# Análisis censo_10_20_mza_res9 en los 4 polígonos del Museo de Ciencias Ambientales

Este notebook reune las sumas totales y porcentajes de grupos de edad, jefatura femenina y hogares con internet en 2010 y 2020 para los cuatro polígonos del Museo de Ciencias Ambientales

In [2]:
import os
import sys

import pandas as pd
import geopandas as gpd
import osmnx as ox
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import folium

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

2023-02-01 23:08:47 Configured OSMnx 1.1.1
2023-02-01 23:08:48 HTTP response caching is on


## Importing and filtering data

In [3]:
#Importing hexres9 with censo 2010 and 2020
gdf = gpd.read_file("../../data/external/to_db/hex_censo_10_20_mza_res9_museocienciasambientales.gpkg")
gdf.head(2)

,hex_id_9,pobtot,pobfem,pobmas,p_0a2,p_0a2_f,p_0a2_m,p_3ymas,p_3ymas_f,p_3ymas_m,...,R_VivHab_20,R_VivDes_20,Z_RVivHab_20,R_VivHab_10,Z_RVivHab_10,Chng_RVivHab_10,DensPob_10_20,city,n_polig,geometry
0,8949ab4b0cfffff,2252.0,1146.0,1106.0,82.0,36.0,38.0,2170.0,1106.0,1064.0,...,0.898928,0.076570,0.588098,0.0,None,0.898928,-48.898518,Guadalajara,2,"POLYGON ((-103.41024 20.72734, -103.40902 20.7..."
1,8949ab4b667ffff,2100.0,1078.0,1022.0,58.0,23.0,21.0,2033.0,1042.0,991.0,...,0.921418,0.041602,0.700936,0.0,None,0.921418,-42.192622,Guadalajara,3,"POLYGON ((-103.39432 20.72775, -103.39310 20.7..."


In [33]:
fields_tokeep = ['n_polig','pobtot', 'pobfem','pobmas','p_0a2','p_3a5','p_6a11','p_12a14','p_15a17','p_18a24','p_60ymas','hogjef_f','vph_inter',
                 '10_pobtot','10_pobfem','10_pobmas','10_p_0a2','10_p_3a5','10_p_6a11','10_p_12a14','10_p_15a17','10_p_18a24','10_p_60ymas','10_hogjef_f','10_vph_inter']
                 
newgdf = gdf[fields_tokeep]
newgdf.head(2)

,n_polig,pobtot,pobfem,pobmas,p_0a2,p_3a5,p_6a11,p_12a14,p_15a17,p_18a24,...,10_pobmas,10_p_0a2,10_p_3a5,10_p_6a11,10_p_12a14,10_p_15a17,10_p_18a24,10_p_60ymas,10_hogjef_f,10_vph_inter
0,2,2252.0,1146.0,1106.0,82.0,99.0,225.0,86.0,98.0,282.0,...,1406.0,179.0,135.0,309.0,129.0,149.0,415.0,356.0,177.0,155.0
1,3,2100.0,1078.0,1022.0,58.0,89.0,183.0,103.0,70.0,221.0,...,1305.0,135.0,130.0,273.0,139.0,126.0,346.0,317.0,209.0,167.0


## Data treatment

In [45]:
newgdf['p_25a59'] = newgdf['pobtot'] - (newgdf['p_0a2']+newgdf['p_3a5']+newgdf['p_6a11']+newgdf['p_12a14']+newgdf['p_15a17']+newgdf['p_18a24']+newgdf['p_60ymas'])
newgdf['10_p_25a59'] = newgdf['10_pobtot'] - (newgdf['10_p_0a2']+newgdf['10_p_3a5']+newgdf['10_p_6a11']+newgdf['10_p_12a14']+newgdf['10_p_15a17']+newgdf['10_p_18a24']+newgdf['10_p_60ymas'])
newgdf

/tmp/ipykernel_8945/1078550627.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newgdf['p_25a59'] = newgdf['pobtot'] - (newgdf['p_0a2']+newgdf['p_3a5']+newgdf['p_6a11']+newgdf['p_12a14']+newgdf['p_15a17']+newgdf['p_18a24']+newgdf['p_60ymas'])
/tmp/ipykernel_8945/1078550627.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newgdf['10_p_25a59'] = newgdf['10_pobtot'] - (newgdf['10_p_0a2']+newgdf['10_p_3a5']+newgdf['10_p_6a11']+newgdf['10_p_12a14']+newgdf['10_p_15a17']+newgdf['10_p_18a24']+newgdf['10_p_60y

,n_polig,pobtot,pobfem,pobmas,p_0a2,p_3a5,p_6a11,p_12a14,p_15a17,p_18a24,...,10_p_3a5,10_p_6a11,10_p_12a14,10_p_15a17,10_p_18a24,10_p_60ymas,10_hogjef_f,10_vph_inter,p_25a59,10_p_25a59
0,2,2252.0,1146.0,1106.0,82.0,99.0,225.0,86.0,98.0,282.0,...,135.0,309.0,129.0,149.0,415.0,356.0,177.0,155.0,972.0,1185.0
1,3,2100.0,1078.0,1022.0,58.0,89.0,183.0,103.0,70.0,221.0,...,130.0,273.0,139.0,126.0,346.0,317.0,209.0,167.0,1020.0,1156.0
2,4,2692.0,1355.0,1337.0,77.0,131.0,243.0,114.0,130.0,300.0,...,136.0,301.0,162.0,150.0,404.0,403.0,242.0,176.0,1192.0,1351.0
3,4,1916.0,995.0,916.0,76.0,87.0,173.0,92.0,104.0,259.0,...,123.0,241.0,116.0,101.0,264.0,239.0,124.0,122.0,817.0,841.0
4,1,359.0,179.0,174.0,16.0,14.0,52.0,14.0,16.0,22.0,...,6.0,18.0,5.0,11.0,8.0,4.0,0.0,0.0,216.0,71.0
5,1,1011.0,513.0,498.0,39.0,28.0,56.0,21.0,25.0,100.0,...,36.0,90.0,43.0,25.0,29.0,15.0,6.0,0.0,722.0,255.0
6,3,1996.0,1042.0,954.0,80.0,97.0,177.0,95.0,95.0,250.0,...,119.0,249.0,127.0,133.0,328.0,253.0,149.0,111.0,903.0,938.0
7,1,273.0,131.0,127.0,0.0,3.0,7.0,5.0,0.0,10.0,...,4.0,11.0,3.0,0.0,3.0,0.0,0.0,0.0,240.0,53.0
8,1,937.0,465.0,472.0,28.0,19.0,42.0,29.0,25.0,64.0,...,25.0,60.0,22.0,25.0,69.0,14.0,16.0,0.0,720.0,218.0
9,1,358.0,170.0,188.0,7.0,16.0,17.0,7.0,13.0,19.0,...,0.0,4.0,0.0,3.0,3.0,0.0,0.0,0.0,274.0,13.0


### Calculate the totals per polygon

In [46]:
fields = {"pobtot":np.nansum,
         "pobfem":np.nansum,
         "pobmas":np.nansum,
         "p_0a2":np.nansum,
         "p_3a5":np.nansum,
         "p_6a11":np.nansum,
         "p_12a14":np.nansum,
         "p_15a17":np.nansum,
         "p_18a24":np.nansum,
         "p_25a59":np.nansum,
         "p_60ymas":np.nansum,
         "hogjef_f":np.nansum,
         "vph_inter":np.nansum,
         "10_pobtot":np.nansum,
         "10_pobfem":np.nansum,
         "10_pobmas":np.nansum,
         "10_p_0a2":np.nansum,
         "10_p_3a5":np.nansum,
         "10_p_6a11":np.nansum,
         "10_p_12a14":np.nansum,
         "10_p_15a17":np.nansum,
         "10_p_18a24":np.nansum,
         "10_p_25a59":np.nansum,
         "10_p_60ymas":np.nansum,
         "10_hogjef_f":np.nansum,
         "10_vph_inter":np.nansum,}

        
summary = newgdf.groupby('n_polig').agg(fields)

### Calculate the percentages per polygon

In [47]:
def calc_pctg(group):
    fields_tocalculate_pctg2020 = ['pobfem','pobmas','p_0a2','p_3a5','p_6a11','p_12a14','p_15a17','p_18a24','p_25a59','p_60ymas']
    fields_tocalculate_pctg2010 = ['10_pobfem','10_pobmas','10_p_0a2','10_p_3a5','10_p_6a11','10_p_12a14','10_p_15a17','10_p_18a24','10_p_25a59','10_p_60ymas']
    
    for field in fields_tocalculate_pctg2020:
        pctg = (group[field] / group['pobtot'])*100
        group['%'+field] = pctg
    
    for field in fields_tocalculate_pctg2010:
        pctg = (group[field] / group['10_pobtot'])*100
        group['%'+field] = pctg
        
    return group
    

summary.groupby('n_polig').apply(calc_pctg)

,pobtot,pobfem,pobmas,p_0a2,p_3a5,p_6a11,p_12a14,p_15a17,p_18a24,p_25a59,...,%10_pobfem,%10_pobmas,%10_p_0a2,%10_p_3a5,%10_p_6a11,%10_p_12a14,%10_p_15a17,%10_p_18a24,%10_p_25a59,%10_p_60ymas
n_polig,,,,,,,,,,,,,,,,,,,,,
1,3678.0,1809.0,1841.0,112.0,112.0,236.0,97.0,96.0,278.0,2666.0,...,47.169811,49.874214,4.528302,6.037736,14.716981,5.786164,5.660377,9.748428,50.691824,2.830189
2,7339.0,3765.0,3574.0,246.0,326.0,627.0,316.0,361.0,906.0,3305.0,...,50.800271,49.087015,5.173580,4.846709,10.538774,5.399008,5.883679,14.055455,41.490081,12.612714
3,5791.0,2991.0,2800.0,198.0,269.0,514.0,266.0,257.0,701.0,2685.0,...,51.275247,48.651039,5.145216,4.983046,11.042312,5.661212,5.720183,13.799204,42.252691,11.396137
4,11950.0,6162.0,5783.0,421.0,560.0,1022.0,507.0,570.0,1393.0,5439.0,...,51.538519,48.461481,4.784008,5.195777,11.012952,5.061017,5.180804,13.109231,43.071049,12.585161
